<a href="https://colab.research.google.com/github/balajisivakumar/stratogrid_Pb/blob/main/Sprint_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install weaviate-client==3.26.0 sentence-transformers protobuf==3.20.3 --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 859.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import weaviate
import json
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

In [7]:
WEAVIATE_URL = "https://jphq0dyssnkkqggxtl12ag.c0.us-east1.gcp.weaviate.cloud"
API_KEY = "MXNUVFN6bkVXY0JlRXBYal9KRVF2YWxFV0M0ZEVyNW1UdTRPRXpPbG81VWxTV2hjT2czcXZiNzZwU2N3PV92MjAw"

client = weaviate.Client(
    url=WEAVIATE_URL,
    auth_client_secret=weaviate.AuthApiKey(API_KEY)
)



#  Create Weaviate schema (delete if exists)
if client.schema.exists("GPTEntry"):
    client.schema.delete_class("GPTEntry")

schema = {
    "class": "GPTEntry",
    "description": "Complete entry from gpt_all_with_id.json",
    "vectorizer": "text2vec-openai",  # change to "none" if using SentenceTransformer
    "properties": [
        {
            "name": "index_no",
            "dataType": ["int"],
            "description": "Field extracted from gpt_all_with_id.json: index_no"
        },
        {
            "name": "summary",
            "dataType": ["text"],
            "description": "Field extracted from gpt_all_with_id.json: summary"
        },
        {
            "name": "keywords",
            "dataType": ["text[]"],
            "description": "Field extracted from gpt_all_with_id.json: keywords"
        },
        {
            "name": "cleaned_text",
            "dataType": ["text"],
            "description": "Field extracted from gpt_all_with_id.json: cleaned_text"
        }
    ]
}

client.schema.create_class(schema)


# Check connection and get cluster class names
if client.is_ready():
    print("✅ Connected:", client.is_ready())
    schema = client.schema.get()
    for cls in schema["classes"]:
        print(f"📦 Class: {cls['class']}")
        for prop in cls["properties"]:
            print(f"   - {prop['name']}: {prop['dataType']}")
else:
    print("❌ Could not connect to Weaviate instance.")


✅ Connected: True
📦 Class: GPTEntry
   - index_no: ['int']
   - summary: ['text']
   - keywords: ['text[]']
   - cleaned_text: ['text']


In [9]:
# Upload and load JSON

with open("/content/gpt_all_with_id.json", "r") as f:
    data = json.load(f)


In [13]:
from tqdm.notebook import tqdm  # Use notebook-friendly version

inserted_count = 0
uuids = []

for item in tqdm(data, desc="Inserting without vector", unit="entry", leave=True):
    obj = {
        "index_no": item.get("index_no"),
        "summary": item.get("summary", ""),
        "keywords": item.get("keywords", []),
        "cleaned_text": item.get("cleaned_text", "")
    }


    inserted_count += 1

print(f"✅ Total rows inserted: {inserted_count}")


Inserting without vector:   0%|          | 0/13915 [00:00<?, ?entry/s]

✅ Total rows inserted: 13915
